In [1]:
import glob
import os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r", encoding='utf-8-sig') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

필요한 라이브러리들을 `import`하고 파일들을 읽어옵니다.

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"[\(?\.!,¿\)]", r"", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

불필요한 공백, 특수문자 등을 없애주고 양 끝에 `<start>`와 `<end>`를 덧붙입니다.

In [3]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(re.findall(r" ", sentence)) >= 14: continue
    if re.search(r'\[', sentence): continue
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

길이가 0인 문장, 중간 공백의 개수가 14개 이상, 즉 단어 개수가 15개 이상인 문장, "\[Hook\]"과 같은 줄을 제외한 뒤 `preprocess_sentence`함수를 적용해 `corpus`에 넣습니다..

In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

텍스트들을 토큰화 하고 텐서에 넣습니다.

In [5]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

텐서의 마지막 토큰을 뺀 것을 `src_input`으로 하고, 첫 번째 토큰을 뺀 것을 `tgt_input`으로 합니다.

In [6]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=12)

`train_test_split()`으로 train 데이터와 validation 데이터를 나눕니다.

In [7]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 300
hidden_size = 2200
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

Keras의 `Model`을 상속하는 모델 클래스를 만듭니다. Model의 `embedding_size`는 너무 낮거나 높으면 잘 안 되는 것 같아 여러 가지를 시도해보면서 조정하였습니다. `hidden_size`또한 `val_loss`를 줄이기 위해 여러 값을 시도해 보았습니다.

In [8]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_delta=0.05, min_lr=1e-5)
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val), callbacks=[reduce_lr, stop_early])

Epoch 1/10
4122/4122 [==============================] - 1088s 258ms/step - loss: 3.3121 - val_loss: 2.6931
Epoch 2/10
4122/4122 [==============================] - 1083s 263ms/step - loss: 2.4721 - val_loss: 2.4249
Epoch 3/10
4122/4122 [==============================] - 1059s 257ms/step - loss: 1.9601 - val_loss: 2.2767
Epoch 4/10
4122/4122 [==============================] - 1064s 258ms/step - loss: 1.5717 - val_loss: 2.2306
Epoch 5/10
4122/4122 [==============================] - 1075s 261ms/step - loss: 1.2303 - val_loss: 2.1836
Epoch 6/10
4122/4122 [==============================] - 1061s 257ms/step - loss: 1.0940 - val_loss: 2.1829
Epoch 7/10
4122/4122 [==============================] - 1061s 257ms/step - loss: 1.0229 - val_loss: 2.1957
Epoch 8/10
4122/4122 [==============================] - 1058s 257ms/step - loss: 1.0160 - val_loss: 2.1978


Model의 `optimizer`를 Adam으로 지정하고 `compile`하여 `fit`으로 학습시킵니다.

In [9]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:

        predict = model(test_tensor) 

        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

모델을 이용해 가사를 만들어내는 `generate_text` 함수를 구현합니다.

In [10]:
print(generate_text(model, tokenizer, init_sentence="<start> you", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> i", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20))

<start> you know i m bad i m bad you know it <end> 
<start> i m the one gon hold you down <end> 
<start> i love you <end> 
<start> i hate the headlines and the weather <end> 


## 회고

**어려웠던 점**: 여러가지를 조정해 보면서 테스트를 해야 하는데 테스트 한 번 돌릴 때마다 1시간 이상 걸리는 점이 제일 어려웠습니다.

**알아낸 것**: 손실이 줄어드는 방향으로 하이퍼파라미터를 조절해가는 게 마치 경사하강법 같군요...

**아직 모호한 점**: `loss`에도 종류가 있는데 차이점이 무엇인지 더 알아봐야 할 것 같습니다. 또 keras layer들의 argument에 대한 이해가 뭔가 두루뭉실하고 어떤 영향을 미치는지 완전히 이해하지 못한 채 결과를 보면서 값만 조정한 느낌이 듭니다.

**평가 지표를 맞추기 위해 시도한 것들**: validation loss를 줄이기 위해 다양한 `embedding_size`와 `hidden_size`를 시도해보았습니다. 또 `ReduceLROnPlateau`라는 `callback`을 추가했습니다.

**다짐**: 좀 더 공부해서 라임을 맞추는 랩 프리스타일 인공지능을 시도해보고 싶습니다.